In [ ]:
# Install FEniCS
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
# Install RBniCS
try:
    import rbnics
except ImportError:
    !pip3 install git+https://github.com/RBniCS/RBniCS.git
    import rbnics
import rbnics.utils.config
assert "dolfin" in rbnics.utils.config.config.get("backends", "required backends")

In [ ]:
from dolfin import *
from mshr import *

In [ ]:
# Create mesh
domain = Rectangle(Point(0., 0.), Point(1., 1.))
mesh = generate_mesh(domain, 24)

In [ ]:
# Create subdomains
subdomains = MeshFunction("size_t", mesh, 2)
subdomains.set_all(1)

In [ ]:
# Create boundaries
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 0.) < DOLFIN_EPS


class Right(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 1.) < DOLFIN_EPS


class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - 0.) < DOLFIN_EPS


class Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1] - 1.) < DOLFIN_EPS


boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)
left = Left()
left.mark(boundaries, 1)
bottom = Bottom()
bottom.mark(boundaries, 1)
top = Top()
top.mark(boundaries, 1)
right = Right()
right.mark(boundaries, 2)

In [ ]:
# Save
File("mesh1.xml") << mesh
File("mesh1_physical_region.xml") << subdomains
File("mesh1_facet_region.xml") << boundaries
XDMFFile("mesh1.xdmf").write(mesh)
XDMFFile("mesh1_physical_region.xdmf").write(subdomains)
XDMFFile("mesh1_facet_region.xdmf").write(boundaries)